In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, f1_score
import numpy as np

# Chargement des données KDDCup99
file_path_kddcup99 = 'KDDCup99.csv'
data = pd.read_csv(file_path_kddcup99)


In [10]:
data.head(50)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,normal
5,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,normal
6,0,tcp,http,SF,212,1940,0,0,0,0,...,69,1.0,0.0,1.00,0.04,0.0,0.0,0.0,0.0,normal
7,0,tcp,http,SF,159,4087,0,0,0,0,...,79,1.0,0.0,0.09,0.04,0.0,0.0,0.0,0.0,normal
8,0,tcp,http,SF,210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.0,0.0,0.0,0.0,normal
9,0,tcp,http,SF,212,786,0,0,0,1,...,99,1.0,0.0,0.12,0.05,0.0,0.0,0.0,0.0,normal


In [3]:
data.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  lnum_compromised             494020 non-null  int64  
 13 

In [ ]:

# Identifier les colonnes catégorielles
categorical_columns = data.select_dtypes(include=['object', 'category']).columns

# Prétraitement des données
if len(categorical_columns) > 0:
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_columns)
        ],
        remainder='passthrough'
    )
else:
    preprocessor = 'passthrough'

# Pipeline de prétraitement et de modèle
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('model', None)
])

# Séparation en caractéristiques (features) et étiquettes (labels)
X = data.drop('label', axis=1)
y = data['label']

# Séparation en données d'entraînement et de test avec échantillonnage stratifié
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)



In [ ]:

# Model definitions
models = {
    "Isolation Forest": IsolationForest(),
    "Local Outlier Factor": LocalOutlierFactor(novelty=True)
}

# Parameters for grid search
param_grid = {
    "Isolation Forest": {"model__n_estimators": [50, 100, 200]},
    "Local Outlier Factor": {"model__n_neighbors": [5, 20, 35]}
}

# Cross-validation setup
cv = KFold(n_splits=5)

# Training and optimizing models
best_models = {}
for name, model in models.items():
    pipeline.set_params(model=model)
    grid_search = GridSearchCV(pipeline, param_grid[name], cv=cv, scoring='f1_macro')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_


In [ ]:

# Model evaluation using F1 score
for name, best_model in best_models.items():
    y_pred = best_model.predict(X_test)
    print(f"Classification report for {name}:")
    print(classification_report(y_test, y_pred))
    print(f"F1 Score for {name}: {f1_score(y_test, y_pred, average='macro')}")

# Select normal data for training novelty detection models
X_train_normal = X_train[y_train == 0]

# Retraining models for novelty detection
models_novelty = {
    "Isolation Forest": IsolationForest(n_estimators=best_models["Isolation Forest"].named_steps['model'].n_estimators),
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=best_models["Local Outlier Factor"].named_steps['model'].n_neighbors, novelty=True)
}

for name, model in models_novelty.items():
    model.fit(X_train_normal)
    y_pred = model.predict(X_test)
    # Adjust predictions for Local Outlier Factor if necessary
    if name == "Local Outlier Factor":
        y_pred = np.where(y_pred == 1, 0, 1)  # Inverting predictions for LOF
    else:
        y_pred = np.where(y_pred == 1, 0, 1)  # Isolation Forest returns -1 for anomalies

    # Evaluation and comparison of performance
    print(f"\nClassification report for novelty detection with {name}:")
    print(classification_report(y_test, y_pred))
    print(f"F1 Score for novelty detection with {name}: {f1_score(y_test, y_pred, average='macro')}")